<a href="https://colab.research.google.com/github/buraktalhaakin/MedSimulator/blob/main/vllmcasegen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- 1. KURULUM (vLLM Engine) ---
# vLLM, şu an dünyadaki en hızlı LLM servis kütüphanesidir.
!pip install -q vllm

import json
import random
import re
from vllm import LLM, SamplingParams
from huggingface_hub import login
from google.colab import files

# Token Girişi
print("Hugging Face Token Giriniz:")
login()

# --- 2. AYARLAR ---
MODEL_ID = "google/medgemma-27b-text-it"

# A100 80GB olduğu için modeli sıkıştırmadan (bfloat16) yükleyebiliriz.
# Bu en yüksek kaliteyi ve hızı verir.
print(f"\n🚀 vLLM Motoru Başlatılıyor... (Model: {MODEL_ID})")

# GPU Kullanım Ayarları
llm = LLM(
    model=MODEL_ID,
    dtype="bfloat16",       # A100'ün ana dili, en hızlısı
    tensor_parallel_size=1, # Tek GPU kullanıyoruz
    gpu_memory_utilization=0.95, # VRAM'in %95'ini kullan (Yaklaşık 76GB)
    max_model_len=4096,     # Context penceresi
    enforce_eager=True,      # CUDA Graph hatasını önlemek için güvenli mod
    max_num_seqs=256,
)

# Üretim Parametreleri
sampling_params = SamplingParams(
    temperature=0.63,
    top_p=0.95,
    max_tokens=4096, # Her vaka için maksimum uzunluk
    stop=["<end_of_turn>"] # Modelin durması gereken yer
)

# --- 3. PROMPT HAZIRLAMA ---
def prompt_olustur(tani):
    return f"""<start_of_turn>user
    Sen klinik simülasyon senaryoları ve vakaları hazırlayan kıdemli bir tıp profesörüsün. Kılavuzlara hakimsin, yaratıcı ve tutarlı vakalar oluşturursun. Kanıta dayalı tıbba hayli önem verirsin. Verilen tanıların hepsinin tıbbi bilgisine, tedavisine hakimsin.

            GÖREV: '{tani}' tanısı için tıp öğrencileri ve doktorları eğitecek zorlukta ve gerçekçilikte bir vaka oluştur. Tutarlılığa maksimum önem ver.

            KRİTİK KURALLAR (Bunlara kesinlikle uy):
            1. **Epidemiyoloji:** Hastanın yaşı ve cinsiyeti, '{tani}' hastalığının gerçek hayattaki insidansına uygun olmalı. Mesleği de bu hastalıkta önemli olabilir. Eğer '{tani}' hastalığı spesifik bir meslekle ilişkili ise o mesleği vakada kullanabilirsin.
            2. **Dil Ayrımı:** - "sikayet" alanı: Hastanın kendi ağzından, basit bir halk diliyle yazılmalı. Hastalar tanılı ve kendi de bildiği hastalıklarından başka bir tıbbi bilgiye sahip olmamalı. (Örn: "Hocam göğsüme bir fil oturdu sanki")
               - Diğer tüm alanlar: Tamamen profesyonel, akademik TIBBİ TERMİNOLOJİ ile yazılmalı.
            3. **Tutarlılık:** Vital bulgular hastalığın şiddetiyle uyumlu olmalı. (Örn: Şok tablosundaysa TA düşük, Nabız yüksek olmalı).
            4. **Laboratuvar:** Her vakada her tetkik için bir cevabın olsun. Patolojik olanlar için ayrıntılı veri ver. Ancak alakasız ise sadece 'normal' diyeceksin. Normal olanlar için ekstra değer yazma. Örneğin kafa travmasında elektrolit sonuçları için sadece 'elektrolitler normal seviyede' diyebilirsin.
            5. **Çıktı:** SADECE JSON formatında yanıt ver. Yorum yapma.

            JSON ŞEMASI:
            {{
                "id": "vaka_ID",
                "gizli_tani": "{tani}",
                "hasta_kimlik": {{
                    "ad_soyad": "Rastgele Türk ismi",
                    "yas": "integer",
                    "cinsiyet": "Erkek/Kadın",
                    "meslek": "Rastgele meslek. Bazen hastalıkla ilişkili uygun meslek de seçilebilir.",
                    "sikayet": "Hastanın ilk cümlesi (Halk ağzı)"
                }},
                "anamnez": {{
                    "sikayet_detaylari": "Tanı ile maksimum derecede uyumlu. Tıbbi dille detaylı anamnez (Süre, karakter, yayılım, eşlik eden semptomlar)",
                    "kronik_hastaliklar": "Varsa ek hastalıklar",
                    "kullandigi_ilaclar": "İlaçlar ve dozları",
                    "tibbi_ozgecmis": "Operasyonlar, alerjiler"
                }},
                "bulgular": {{
                    "fizik_muayene": "Sistemik muayene (Genel durum, Vital Bulgular: TA, Nabız, Ateş, SpO2, Solunum sayısı mutlaka olsun)",
                    "laboratuvar": "Laboratuvar tetkik sonuçları (Birimleriyle)",
                    "goruntuleme": "Görüntüleme ve EKG raporu sonucu"
                }}
            }}<end_of_turn>
<start_of_turn>model
            """

# --- 4. TOPLU ÜRETİM (BATCH PROCESSING) ---
tanilar = [
    # --- ACİL VE KRİTİK DURUMLAR ---
    "Akut Pankreatit",
    "Diyabetik Ketoasidoz (DKA)",
    "Toplum Kökenli Pnömoni",
    "Akut Piyelonefrit",
    "Pulmoner Emboli",
    "Hipertansif Acil",
    "KOAH Alevlenmesi",
    "Tansiyon Pnömotoraks",
    "Subaraknoid Kanama",
    "Akut Apandisit",
    "Akut Böbrek Hasarı",
    "Kafa Travması",
    "Serebrovasküler Olay (SVO - İnme)",
    "Miyokard Enfarktüsü (USAP-NSTEMI-STEMI)",
    "Anafilaksi",
    "Aort Diseksiyonu",
    "Akut Pulmoner Ödem",
    "Akut Karaciğer Yetmezliği",
    "Akut Menenjit",
    "Akut Kolanjit",
    "Akut Divertikülit",
    "Akut Hipotansiyon (Şok)",
    "Ektopik Gebelik", # Yeni (Hayati)
    "Tetanos Şüpheli Yaralanma", # Yeni

    # --- KARDİYOLOJİ VE RİTİM ---
    "Atriyal Fibrilasyon",
    "Supraventriküler Taşikardi (SVT)",
    "Bradikardi",
    "Ventriküler Taşikardi",
    "Ventriküler Fibrilasyon",
    "Perikardit",
    "Derin Ven Trombozu (DVT)",

    # --- METABOLİZMA VE ELEKTROLİT ---
    "Hipokalemi",
    "Hiperkalemi",
    "Hiponatremi",
    "Hipernatremi",
    "Tirotoksikoz",
    "Dislipidemi",
    "Gut Atağı",

    # --- ENFEKSİYON VE KBB ---
    "Akut Nazofarenjit (ÜSYE)",
    "Akut Tonsillofarenjit",
    "Akut Sinüzit",
    "İnfluenza",
    "Akut Otitis Media",
    "Akut Otitis Eksterna",
    "Enfeksiyöz Mononükleoz", # Yeni
    "Buşon (Kulak Kiri)", # Yeni

    # --- SOLUNUM VE ALERJİ ---
    "Akut Astım Atağı",
    "Alerjik Rinit",
    "Akut Ürtiker",

    # --- GASTROINTESTINAL ---
    "Gastroözofageal Reflü",
    "Akut Gastrit",
    "Akut Gastroenterit",
    "Akut Kolesistit",
    "Hemoroidal Hastalık",
    "Besin Zehirlenmesi", # Yeni

    # --- KAS İSKELET VE ROMATOLOJİ ---
    "Mekanik Bel Ağrısı",
    "Lomber Disk Hernisi",
    "Servikal Disk Hernisi", # Yeni
    "Osteoartrit",
    "Fibromiyalji",
    "Ayak Bileği Burkulması",
    "Kemik Kırığı (Ekstremite)",
    "Plantar Fasiit (Topuk Dikeni)", # Yeni

    # --- NÖROLOJİ ---
    "Migren",
    "Gerilim Tipi Baş Ağrısı",
    "BPPV (Vertigo)",
    "Vestibüler Nörit",
    "Epilepsi Nöbeti",
    "Karpal Tünel Sendromu",
    "Bell Paralizisi (Yüz Felci)",

    # --- DERMATOLOJİ (CİLDİYE) ---
    "Uyuz (Scabies)", # Yeni (Çok Önemli)
    "Zona (Herpes Zoster)", # Yeni
    "Atopik Dermatit (Egzama)", # Yeni
    "Tinea Pedis (Ayak Mantarı)", # Yeni
    "Akne Vulgaris", # Yeni
    "Selülit ve Erizipel", # Yeni
    "Kedi/Köpek Isırması", # Yeni
    "Yanıklar (1. ve 2. Derece)",

    # --- ÜROLOJİ VE NEFROLOJİ ---
    "Akut Sistit",
    "Renal Kolik (Böbrek Taşı)",
    "Akut Glomerülonefrit",
    "Benign Prostat Hiperplazisi (BPH)", # Yeni
    "Varikosel", # Yeni
    "Üretrit (Gonore/Klamidya Şüphesi)", # Yeni

    # --- KADIN HASTALIKLARI ---
    "Dismenore",
    "Vajinit (Kandidal/Bakteriyel)", # Yeni
    "Mastit", # Yeni
    "Gebelik Hiperemezisi", # Yeni

    # --- GÖZ ---
    "Konjonktivit",
    "Korneal Yabancı Cisim", # Yeni
    "Hordeolum (Arpacık)", # Yeni

    # --- PSİKİYATRİ ---
    "Panik Atak",
    "Majör Depresif Bozukluk", # Yeni
    "Yaygın Anksiyete Bozukluğu", # Yeni
    "İnsomnia (Uykusuzluk)", # Yeni

    # --- PEDİATRİ (İsteğe Bağlı) ---
    "Febril Konvülsiyon", # Yeni
    "Krup Sendromu", # Yeni
    "El-Ayak-Ağız Hastalığı" # Yeni
]

# Prompt Listesini Hazırla (Her tanıdan 100 adet)
all_prompts = []
tani_meta = [] # Hangi promptun hangi tanıya ait olduğunu tutar

ADET_BASI = 25

print(f"\n⚡ {len(tanilar) * ADET_BASI} adet vaka için promptlar hazırlanıyor...")
for tani in tanilar:
    for _ in range(ADET_BASI):
        all_prompts.append(prompt_olustur(tani))
        tani_meta.append(tani)

# --- MOTORU ATEŞLE ---
print(f"BATCH GENERATION BAŞLIYOR...")
outputs = llm.generate(all_prompts, sampling_params)

# --- 5. SONUÇLARI İŞLE ---
vaka_veritabani = []
global_sayac = 1

def extract_json_clean(text):
    try:
        text = re.sub(r'```json\s*', '', text)
        text = re.sub(r'```', '', text)
        start = text.find('{')
        end = text.rfind('}') + 1
        if start != -1 and end != -1:
            return json.loads(text[start:end])
    except:
        return None

print("\n✅ Üretim tamamlandı. JSON formatına dönüştürülüyor...")

for i, output in enumerate(outputs):
    generated_text = output.outputs[0].text
    tani_ismi = tani_meta[i]

    data = extract_json_clean(generated_text)

    if data:
        data["id"] = f"vaka_{global_sayac:03d}"
        data["gizli_tani"] = tani_ismi
        vaka_veritabani.append(data)
        global_sayac += 1
    else:
        print(f"⚠️ Format Hatası (Index {i} - {tani_ismi})")

# Karıştır
random.shuffle(vaka_veritabani)

# Kaydet
dosya_adi = "medsim_vllm_ultra_fast.json"
with open(dosya_adi, "w", encoding="utf-8") as f:
    json.dump(vaka_veritabani, f, ensure_ascii=False, indent=4)

print(f"\n🎉 İŞLEM BİTTİ! Toplam {len(vaka_veritabani)} vaka üretildi.")
print(f"Dosya: {dosya_adi}")
files.download(dosya_adi)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 136.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 108.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 152.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.


🚀 vLLM Motoru Başlatılıyor... (Model: google/medgemma-27b-text-it)
INFO 12-10 15:43:57 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'seed': None, 'max_model_len': 4096, 'gpu_memory_utilization': 0.95, 'max_num_seqs': 256, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'google/medgemma-27b-text-it'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

WARNING 12-10 15:43:58 [arg_utils.py:1175] `seed=None` is equivalent to `seed=0` in V1 Engine. You will no longer be allowed to pass `None` in v0.13.
INFO 12-10 15:44:16 [model.py:637] Resolved architecture: Gemma3ForCausalLM
INFO 12-10 15:44:16 [model.py:1750] Using max model len 4096
INFO 12-10 15:44:19 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-10 15:44:19 [vllm.py:601] Enforce eager set, overriding optimization level to -O0
INFO 12-10 15:44:19 [vllm.py:707] Cudagraph is disabled under eager mode


...

Adding requests:   0%|          | 0/2450 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/2450 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…


✅ Üretim tamamlandı. JSON formatına dönüştürülüyor...
⚠️ Format Hatası (Index 1 - Akut Pankreatit)
⚠️ Format Hatası (Index 2 - Akut Pankreatit)
⚠️ Format Hatası (Index 9 - Akut Pankreatit)
⚠️ Format Hatası (Index 14 - Akut Pankreatit)
⚠️ Format Hatası (Index 17 - Akut Pankreatit)
⚠️ Format Hatası (Index 18 - Akut Pankreatit)
⚠️ Format Hatası (Index 24 - Akut Pankreatit)
⚠️ Format Hatası (Index 33 - Diyabetik Ketoasidoz (DKA))
⚠️ Format Hatası (Index 36 - Diyabetik Ketoasidoz (DKA))
⚠️ Format Hatası (Index 40 - Diyabetik Ketoasidoz (DKA))
⚠️ Format Hatası (Index 41 - Diyabetik Ketoasidoz (DKA))
⚠️ Format Hatası (Index 50 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 52 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 54 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 56 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 61 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 63 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 67 - Toplum Kökenli Pnömoni)
⚠️ Format Hatası (Index 70 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# --- 1. KURULUM ---
!pip install -q vllm

import json
import random
import re
import os
from vllm import LLM, SamplingParams
from huggingface_hub import login
from google.colab import files

print("Hugging Face Token Giriniz:")
login()

# --- 2. AYARLAR ---
MODEL_ID = "google/medgemma-27b-text-it"
MEVCUT_DOSYA = "/content/medsim_case_v2.0.json" # Eğer farklı isimle kaydettiysen burayı düzelt
HEDEF_SAYI = 25 # Her hastalıktan kaç tane olmalıydı?

# --- 3. MEVCUT DOSYAYI YÜKLEME VE ANALİZ ---
print(f"\n📂 Mevcut dosya analiz ediliyor: {MEVCUT_DOSYA}")

# Eğer dosya Colab'da yoksa yüklemeni ister
if not os.path.exists(MEVCUT_DOSYA):
    print(f"⚠️ '{MEVCUT_DOSYA}' bulunamadı. Lütfen dosyayı sol menüden yükleyin.")
    # Dosya yükleme penceresini açar (Manuel yükle)
    uploaded = files.upload()
    MEVCUT_DOSYA = list(uploaded.keys())[0]

with open(MEVCUT_DOSYA, 'r', encoding='utf-8') as f:
    mevcut_veritabani = json.load(f)

# Hangi hastalıktan kaç tane var sayalım
sayac = {}
for vaka in mevcut_veritabani:
    tani = vaka.get("gizli_tani")
    if tani:
        sayac[tani] = sayac.get(tani, 0) + 1

# --- 4. EKSİK LİSTESİ ÇIKARMA ---
# Tam tanı listesi (Senin uzun listen)
tam_tanilar = [
    # --- ACİL VE KRİTİK DURUMLAR ---
    "Akut Pankreatit",
    "Diyabetik Ketoasidoz (DKA)",
    "Toplum Kökenli Pnömoni",
    "Akut Piyelonefrit",
    "Pulmoner Emboli",
    "Hipertansif Acil",
    "KOAH Alevlenmesi",
    "Tansiyon Pnömotoraks",
    "Subaraknoid Kanama",
    "Akut Apandisit",
    "Akut Böbrek Hasarı",
    "Kafa Travması",
    "Serebrovasküler Olay (SVO - İnme)",
    "Miyokard Enfarktüsü (USAP-NSTEMI-STEMI)",
    "Anafilaksi",
    "Aort Diseksiyonu",
    "Akut Pulmoner Ödem",
    "Akut Karaciğer Yetmezliği",
    "Akut Menenjit",
    "Akut Kolanjit",
    "Akut Divertikülit",
    "Akut Hipotansiyon (Şok)",
    "Ektopik Gebelik", # Yeni (Hayati)
    "Tetanos Şüpheli Yaralanma", # Yeni

    # --- KARDİYOLOJİ VE RİTİM ---
    "Atriyal Fibrilasyon",
    "Supraventriküler Taşikardi (SVT)",
    "Bradikardi",
    "Ventriküler Taşikardi",
    "Ventriküler Fibrilasyon",
    "Perikardit",
    "Derin Ven Trombozu (DVT)",

    # --- METABOLİZMA VE ELEKTROLİT ---
    "Hipokalemi",
    "Hiperkalemi",
    "Hiponatremi",
    "Hipernatremi",
    "Tirotoksikoz",
    "Dislipidemi",
    "Gut Atağı",

    # --- ENFEKSİYON VE KBB ---
    "Akut Nazofarenjit (ÜSYE)",
    "Akut Tonsillofarenjit",
    "Akut Sinüzit",
    "İnfluenza",
    "Akut Otitis Media",
    "Akut Otitis Eksterna",
    "Enfeksiyöz Mononükleoz", # Yeni
    "Buşon (Kulak Kiri)", # Yeni

    # --- SOLUNUM VE ALERJİ ---
    "Akut Astım Atağı",
    "Alerjik Rinit",
    "Akut Ürtiker",

    # --- GASTROINTESTINAL ---
    "Gastroözofageal Reflü",
    "Akut Gastrit",
    "Akut Gastroenterit",
    "Akut Kolesistit",
    "Hemoroidal Hastalık",
    "Besin Zehirlenmesi", # Yeni

    # --- KAS İSKELET VE ROMATOLOJİ ---
    "Mekanik Bel Ağrısı",
    "Lomber Disk Hernisi",
    "Servikal Disk Hernisi", # Yeni
    "Osteoartrit",
    "Fibromiyalji",
    "Ayak Bileği Burkulması",
    "Kemik Kırığı (Ekstremite)",
    "Plantar Fasiit (Topuk Dikeni)", # Yeni

    # --- NÖROLOJİ ---
    "Migren",
    "Gerilim Tipi Baş Ağrısı",
    "BPPV (Vertigo)",
    "Vestibüler Nörit",
    "Epilepsi Nöbeti",
    "Karpal Tünel Sendromu",
    "Bell Paralizisi (Yüz Felci)",

    # --- DERMATOLOJİ (CİLDİYE) ---
    "Uyuz (Scabies)", # Yeni (Çok Önemli)
    "Zona (Herpes Zoster)", # Yeni
    "Atopik Dermatit (Egzama)", # Yeni
    "Tinea Pedis (Ayak Mantarı)", # Yeni
    "Akne Vulgaris", # Yeni
    "Selülit ve Erizipel", # Yeni
    "Kedi/Köpek Isırması", # Yeni
    "Yanıklar (1. ve 2. Derece)",

    # --- ÜROLOJİ VE NEFROLOJİ ---
    "Akut Sistit",
    "Renal Kolik (Böbrek Taşı)",
    "Akut Glomerülonefrit",
    "Benign Prostat Hiperplazisi (BPH)", # Yeni
    "Varikosel", # Yeni
    "Üretrit (Gonore/Klamidya Şüphesi)", # Yeni

    # --- KADIN HASTALIKLARI ---
    "Dismenore",
    "Vajinit (Kandidal/Bakteriyel)", # Yeni
    "Mastit", # Yeni
    "Gebelik Hiperemezisi", # Yeni

    # --- GÖZ ---
    "Konjonktivit",
    "Korneal Yabancı Cisim", # Yeni
    "Hordeolum (Arpacık)", # Yeni

    # --- PSİKİYATRİ ---
    "Panik Atak",
    "Majör Depresif Bozukluk", # Yeni
    "Yaygın Anksiyete Bozukluğu", # Yeni
    "İnsomnia (Uykusuzluk)", # Yeni

    # --- PEDİATRİ (İsteğe Bağlı) ---
    "Febril Konvülsiyon", # Yeni
    "Krup Sendromu", # Yeni
    "El-Ayak-Ağız Hastalığı" # Yeni
]

eksik_gorevler = []
print("\n🔍 Eksikler Tespit Ediliyor...")

for tani in tam_tanilar:
    mevcut = sayac.get(tani, 0)
    eksik = HEDEF_SAYI - mevcut
    if eksik > 0:
        print(f"  -> {tani}: {mevcut} var, {eksik} EKSİK.")
        for _ in range(eksik):
            eksik_gorevler.append(tani)

if not eksik_gorevler:
    print("\n✅ HİÇ EKSİK YOK! Veri setin tam.")
else:
    print(f"\n⚡ Toplam {len(eksik_gorevler)} adet eksik vaka tamamlanacak.")

    # --- 5. MOTORU BAŞLAT VE ÜRET (DAHA YÜKSEK TOKEN LİMİTİYLE) ---
    print(f"🚀 vLLM Motoru Başlatılıyor (Kurtarma Modu)...")

    llm = LLM(
        model=MODEL_ID,
        dtype="bfloat16",
        tensor_parallel_size=1,
        gpu_memory_utilization=0.98,
        max_model_len=8192, # Context'i artırdık
        enforce_eager=True
    )

    # Token Limitini Artırıyoruz ki yine kesilmesin
    sampling_params = SamplingParams(
        temperature=0.7,
        top_p=0.95,
        max_tokens=8192, # Kesilmeyi önlemek için artırdık
        stop=["<end_of_turn>"]
    )

    def prompt_olustur(tani):
        return f"""<start_of_turn>user
    Sen klinik simülasyon senaryoları ve vakaları hazırlayan kıdemli bir tıp profesörüsün. Kılavuzlara hakimsin, yaratıcı ve tutarlı vakalar oluşturursun. Kanıta dayalı tıbba hayli önem verirsin. Verilen tanıların hepsinin tıbbi bilgisine, tedavisine hakimsin.

            GÖREV: '{tani}' tanısı için tıp öğrencileri ve doktorları eğitecek zorlukta ve gerçekçilikte bir vaka oluştur. Tutarlılığa maksimum önem ver.

            KRİTİK KURALLAR (Bunlara kesinlikle uy):
            1. **Epidemiyoloji:** Hastanın yaşı ve cinsiyeti, '{tani}' hastalığının gerçek hayattaki insidansına uygun olmalı. Mesleği de bu hastalıkta önemli olabilir. Eğer '{tani}' hastalığı spesifik bir meslekle ilişkili ise o mesleği vakada kullanabilirsin.
            2. **Dil Ayrımı:** - "sikayet" alanı: Hastanın kendi ağzından, basit bir halk diliyle yazılmalı. Hastalar tanılı ve kendi de bildiği hastalıklarından başka bir tıbbi bilgiye sahip olmamalı. (Örn: "Hocam göğsüme bir fil oturdu sanki")
               - Diğer tüm alanlar: Tamamen profesyonel, akademik TIBBİ TERMİNOLOJİ ile yazılmalı.
            3. **Tutarlılık:** Vital bulgular hastalığın şiddetiyle uyumlu olmalı. (Örn: Şok tablosundaysa TA düşük, Nabız yüksek olmalı).
            4. **Laboratuvar:** Her vakada her tetkik için bir cevabın olsun. Patolojik olanlar için ayrıntılı veri ver. Ancak alakasız ise sadece 'normal' diyeceksin. Normal olanlar için ekstra değer yazma. Örneğin kafa travmasında elektrolit sonuçları için sadece 'elektrolitler normal seviyede' diyebilirsin.
            5. **Çıktı:** SADECE JSON formatında yanıt ver. Yorum yapma.

            JSON ŞEMASI:
            {{
                "id": "vaka_ID",
                "gizli_tani": "{tani}",
                "hasta_kimlik": {{
                    "ad_soyad": "Rastgele Türk ismi",
                    "yas": "integer",
                    "cinsiyet": "Erkek/Kadın",
                    "meslek": "Rastgele meslek. Bazen hastalıkla ilişkili uygun meslek de seçilebilir.",
                    "sikayet": "Hastanın ilk cümlesi (Halk ağzı)"
                }},
                "anamnez": {{
                    "sikayet_detaylari": "Tanı ile maksimum derecede uyumlu. Tıbbi dille detaylı anamnez (Süre, karakter, yayılım, eşlik eden semptomlar)",
                    "kronik_hastaliklar": "Varsa ek hastalıklar",
                    "kullandigi_ilaclar": "İlaçlar ve dozları",
                    "tibbi_ozgecmis": "Operasyonlar, alerjiler"
                }},
                "bulgular": {{
                    "fizik_muayene": "Sistemik muayene (Genel durum, Vital Bulgular: TA, Nabız, Ateş, SpO2, Solunum sayısı mutlaka olsun)",
                    "laboratuvar": "Laboratuvar tetkik sonuçları (Birimleriyle)",
                    "goruntuleme": "Görüntüleme ve EKG raporu sonucu"
                }}
            }}<end_of_turn>
<start_of_turn>model
            """

    prompts = [prompt_olustur(t) for t in eksik_gorevler]

    print("🔥 EKSİKLER TAMAMLANIYOR...")
    outputs = llm.generate(prompts, sampling_params)

    # --- 6. BİRLEŞTİRME VE KAYIT ---

    def extract_json_fast(text):
        try:
            text = re.sub(r'```json\s*', '', text).replace('```', '').strip()
            s = text.find('{')
            e = text.rfind('}') + 1
            if s != -1 and e != -1:
                return json.loads(text[s:e])
        except:
            return None
        return None

    basarili_eklenen = 0

    for i, output in enumerate(outputs):
        data = extract_json_fast(output.outputs[0].text)
        if data:
            # ID çakışmasını önlemek için listenin sonuna ekle
            data["id"] = f"vaka_{len(mevcut_veritabani)+1:04d}"
            # Tanı garantisi
            if "gizli_tani" not in data:
                data["gizli_tani"] = eksik_gorevler[i]

            mevcut_veritabani.append(data)
            basarili_eklenen += 1
        else:
            print(f"⚠️ Yine hata oldu: {eksik_gorevler[i]}")

    print(f"\n🎉 TAMAMLANDI! {basarili_eklenen} yeni vaka eklendi.")
    print(f"Toplam Veritabanı Boyutu: {len(mevcut_veritabani)}")

    # Karıştır ve Kaydet
    random.shuffle(mevcut_veritabani)

    dosya_adi = "medsim_case_v2.1.json"
    with open(dosya_adi, "w", encoding="utf-8") as f:
        json.dump(mevcut_veritabani, f, ensure_ascii=False, indent=4)

    files.download(dosya_adi)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.5/466.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.0/355.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/10


📂 Mevcut dosya analiz ediliyor: /content/medsim_case_v2.0.json

🔍 Eksikler Tespit Ediliyor...
  -> Pulmoner Emboli: 24 var, 1 EKSİK.
  -> Hipertansif Acil: 24 var, 1 EKSİK.
  -> Tansiyon Pnömotoraks: 19 var, 6 EKSİK.
  -> Akut Böbrek Hasarı: 22 var, 3 EKSİK.
  -> Miyokard Enfarktüsü (USAP-NSTEMI-STEMI): 17 var, 8 EKSİK.
  -> Aort Diseksiyonu: 23 var, 2 EKSİK.
  -> Akut Pulmoner Ödem: 24 var, 1 EKSİK.
  -> Akut Kolanjit: 24 var, 1 EKSİK.
  -> Akut Hipotansiyon (Şok): 23 var, 2 EKSİK.
  -> Atriyal Fibrilasyon: 19 var, 6 EKSİK.
  -> Bradikardi: 15 var, 10 EKSİK.
  -> Ventriküler Taşikardi: 17 var, 8 EKSİK.
  -> Ventriküler Fibrilasyon: 21 var, 4 EKSİK.
  -> Hipokalemi: 22 var, 3 EKSİK.
  -> Hiperkalemi: 24 var, 1 EKSİK.
  -> Hiponatremi: 24 var, 1 EKSİK.
  -> İnfluenza: 24 var, 1 EKSİK.
  -> Akut Otitis Media: 23 var, 2 EKSİK.
  -> Alerjik Rinit: 24 var, 1 EKSİK.
  -> Akut Ürtiker: 24 var, 1 EKSİK.
  -> Akut Gastroenterit: 16 var, 9 EKSİK.
  -> Besin Zehirlenmesi: 22 var, 3 EKSİK.
  -> E

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/931 [00:00<?, ?B/s]

WARNING 12-10 18:42:09 [arg_utils.py:1175] `seed=None` is equivalent to `seed=0` in V1 Engine. You will no longer be allowed to pass `None` in v0.13.
INFO 12-10 18:42:28 [model.py:637] Resolved architecture: Gemma3ForCausalLM
INFO 12-10 18:42:28 [model.py:1750] Using max model len 8192
INFO 12-10 18:42:31 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-10 18:42:31 [vllm.py:601] Enforce eager set, overriding optimization level to -O0
INFO 12-10 18:42:31 [vllm.py:707] Cudagraph is disabled under eager mode


...

Adding requests:   0%|          | 0/87 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/87 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

⚠️ Yine hata oldu: Pulmoner Emboli
⚠️ Yine hata oldu: Tansiyon Pnömotoraks
⚠️ Yine hata oldu: Tansiyon Pnömotoraks
⚠️ Yine hata oldu: Tansiyon Pnömotoraks
⚠️ Yine hata oldu: Tansiyon Pnömotoraks
⚠️ Yine hata oldu: Tansiyon Pnömotoraks
⚠️ Yine hata oldu: Akut Böbrek Hasarı
⚠️ Yine hata oldu: Akut Böbrek Hasarı
⚠️ Yine hata oldu: Miyokard Enfarktüsü (USAP-NSTEMI-STEMI)
⚠️ Yine hata oldu: Miyokard Enfarktüsü (USAP-NSTEMI-STEMI)
⚠️ Yine hata oldu: Miyokard Enfarktüsü (USAP-NSTEMI-STEMI)
⚠️ Yine hata oldu: Miyokard Enfarktüsü (USAP-NSTEMI-STEMI)
⚠️ Yine hata oldu: Akut Kolanjit
⚠️ Yine hata oldu: Akut Hipotansiyon (Şok)
⚠️ Yine hata oldu: Atriyal Fibrilasyon
⚠️ Yine hata oldu: Atriyal Fibrilasyon
⚠️ Yine hata oldu: Atriyal Fibrilasyon
⚠️ Yine hata oldu: Atriyal Fibrilasyon
⚠️ Yine hata oldu: Atriyal Fibrilasyon
⚠️ Yine hata oldu: Atriyal Fibrilasyon
⚠️ Yine hata oldu: Bradikardi
⚠️ Yine hata oldu: Bradikardi
⚠️ Yine hata oldu: Bradikardi
⚠️ Yine hata oldu: Bradikardi
⚠️ Yine hata oldu: Brad

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>